In [1]:
import os
import radiomics
import pandas as pd
import numpy as np
import SimpleITK as sitk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
para_path_CT = "./Params/Params.yaml"
extractor_CT = radiomics.featureextractor.RadiomicsFeatureExtractor(para_path_CT)
df_CT = pd.DataFrame()
path = "./Pleural_Radiomics/1"

In [ ]:
for folder in os.listdir(path):
    df_CT_ID = pd.DataFrame([str(folder)], columns =["ID"])
    label = pd.DataFrame([1],columns=["label"])
    dataDir = os.path.join(path, folder)
    imageName = "CT.nii.gz"
    maskName = "CT_M.nii.gz"
    imagePath = os.path.join(dataDir, imageName)
    maskPath = os.path.join(dataDir , maskName)
    featureVector_CT = extractor_CT.execute(imagePath, maskPath)  
    df_add_CT = pd.DataFrame.from_dict(featureVector_CT.values()).T
    df_add_CT.columns = featureVector_CT.keys()
    col1 = pd.concat([df_CT_ID,label],axis=1)
    col2 = pd.concat([col1,df_add_CT],axis=1)
    df_CT = pd.concat([df_CT, col2])


In [ ]:
df_CT.to_csv("./output/extract_CT.csv")

In [ ]:
para_path = "./Params/Params_2.yaml"
extractor = radiomics.featureextractor.RadiomicsFeatureExtractor(para_path)
df_PET = pd.DataFrame()


In [ ]:
for folder in os.listdir(path):
    df_PET_ID = pd.DataFrame([str(folder)], columns =["ID"])
    label = pd.DataFrame([1],columns=["label"])
    dataDir = os.path.join(path, folder)
    imageName =  "PET.nii.gz"
    maskName = "PET_M_R.nii.gz"
    imagePath = os.path.join(dataDir, imageName)
    maskPath = os.path.join(dataDir , maskName)
    print(folder) 
    featureVector = extractor.execute(imagePath, maskPath) 
    df_add = pd.DataFrame.from_dict(featureVector.values()).T
    df_add.columns = featureVector.keys()
    col1 = pd.concat([df_PET_ID,label],axis=1)
    col2 = pd.concat([col1,df_add],axis=1)
    df_PET = pd.concat([df_PET, col2])
columns_list = df_PET.columns.to_list()
pet_columns = ['PET_'+str(i) for i in columns_list]
df_PET.columns = pet_columns


In [ ]:
df_PET = df_PET.rename(columns={'PET_ID':'ID'})
df_PET.to_csv("./output/extract_PET.csv")

In [ ]:
data = pd.read_csv("./output/data.csv")
X_n = data.loc[:,"AGE":]
# X_n.head(5)
colNames = X_n.columns
X_n = X_n.astype(np.float64)
X_n = X_n.fillna(X_n.mean())
X_n = StandardScaler().fit_transform(X_n)
X_n = pd.DataFrame(X_n)
X_n.columns = colNames
data_n = pd.concat([data.loc[:,"label":"Pathology"],X_n],axis=1)
data_n.to_csv("./output/data_n.csv")